<a href="https://colab.research.google.com/github/hjae0520/class2022Spring/blob/main/requests_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

과제:API, Gradio, 미세먼지 말고, 겹치지 않게
다음 주 수요일 수업전까지
파일명 20220328.ipynb

API: 개발자의 입장에서 특정 프로그램을 내 프로그램으로 가져와서 쓸 수 있게 해주는 것, 프로그래밍의 부품
ex. 파파고를 내 프로그램으로 가져와서 쓸 때, 자동번역 api 등등
API를 가져오는 것이 requests
api 유형은 rest를 사용
데이터포맷은 크게 jason or xml을 활용


In [ ]:
!pip install gradio
import gradio as gr
import requests
import pandas as pd #pandas는 데이터 프레임을 만드는 함수

In [ ]:
# requests.get(요청메세지 = 주소)

In [ ]:
URL = ('http://apis.data.go.kr/B552584/ArpltnStatsSvc/'    # 메뉴얼 상 서비스 URL, rest api
      'getCtprvnMesureSidoLIst?'                          # 사용하려는 상세기능명 + ?
      'sidoName=서울'                                     # 요청 메세지 명세(세부 항목) ex.시도 명 = ~~ & 검색 시간 = ~~  & 항수 = ~~~ 등 메뉴얼에 다 나와 있음
      '&searchCondition=hour'
      '&pageNo=1'
      '&numOfRows=100'
      '&returnType=json'                                  # returntype - 중요
      '&serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D')       # 서비스 키, 여기까지가 api 사용끝
response = requests.get(URL)

In [ ]:
response # 200 --> response code, 200은 잘 처리가 되었다는 뜻

<Response [200]>

#### http://jsonviewer.stack.hu/ (copy json here to view json)

In [ ]:
r = response.json() # response에 있는 내용 확인 방법, r 출력값을 복사해서 위 사이트에 붙여넣기 하면 Jason의 구조를 확인할 수 있음 Jason -> response -> body -> items -> 0~24...
r

In [ ]:
type (r) # dict -> {key : values} -> z {key} = value 활용

dict

In [ ]:
items = r['response']['body']['items'] # list로 되어 있는데 그 안을 시, 구별 dict들이 채우고 있음, items [0]: 맨 처음인 강남구 dict가 출력
items[0]                               

{'cityName': '강남구',
 'cityNameEng': 'Gangnam-gu',
 'coValue': '0.3',
 'dataGubun': '',
 'dataTime': '2022-03-23 17:00',
 'districtCode': '02',
 'districtNumSeq': '001',
 'itemCode': '',
 'khaiValue': '',
 'no2Value': '0.023',
 'numOfRows': '10',
 'o3Value': '0.036',
 'pageNo': '1',
 'pm10Value': '32',
 'pm25Value': '16',
 'resultCode': '',
 'resultMsg': '',
 'returnType': 'json',
 'searchCondition': '',
 'serviceKey': '',
 'sidoName': '서울',
 'so2Value': '0.003',
 'totalCount': ''}

In [ ]:
items = r['response']['body']['items'] # response 중에 body 중에 items을 items이라는 변수로 받겠다, 이걸 pandas로 data frame을 만드는 것이 목표

cityName = [item['cityName'] for item in items]           # list comprehension을 활용, in items (items 중에서), for item (item이라는 변수를 하나씩 받을 건데), 그것들에 cityName을 붙일 거다
pm10Value = [item['pm10Value'] for item in items]         # 
pm25Value = [item['pm25Value'] for item in items]
so2Value = [item['so2Value'] for item in items]
coValue = [item['coValue'] for item in items]
o3Value = [item['o3Value'] for item in items]
no2Value = [item['no2Value'] for item in items]

cityName = pd.Series(cityName)     # key값을 데이터 프레임의 x축 값으로 대입,하나의 pd.series는 한 열의 내용을 채울 수 있음
pm10Value = pd.Series(pm10Value)   
pm25Value = pd.Series(pm25Value)
so2Value = pd.Series(so2Value)
coValue = pd.Series(coValue)
o3Value = pd.Series(o3Value)
no2Value = pd.Series(no2Value)

df = pd.DataFrame({ 'cityName': cityName, 'pm10Value': pm10Value, 'pm25Value': pm25Value, 'so2Value': so2Value, 'coValue': coValue, 'o3Value': o3Value, 'no2Value': no2Value})
# '열의 x축' : 내용값

In [ ]:
df

In [ ]:
def test(sidoName):                                       # gradio는 함수를 전제로 하기 때문에 함수를 정의해야 함, 위의 도시명을 입력하면 해당 도시의 df가 출력되는 식
  URL = ('http://apis.data.go.kr/B552584/ArpltnStatsSvc/'
      'getCtprvnMesureSidoLIst?'
      'sidoName='+sidoName+                       # +sidoname+
      '&searchCondition=hour'
      '&pageNo=1'
      '&numOfRows=100'
      '&returnType=json'
      '&serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D')
  
  response = requests.get(URL)
  r = response.json()
  items = r['response']['body']['items']

  cityName = [item['cityName'] for item in items]
  pm10Value = [item['pm10Value'] for item in items]
  pm25Value = [item['pm25Value'] for item in items]
  so2Value = [item['so2Value'] for item in items]
  coValue = [item['coValue'] for item in items]
  o3Value = [item['o3Value'] for item in items]
  no2Value = [item['no2Value'] for item in items]

  pd_cityName = pd.Series(cityName)
  pd_pm10Value = pd.Series(pm10Value)   
  pd_pm25Value = pd.Series(pm25Value)
  so2Value = pd.Series(so2Value)
  coValue = pd.Series(coValue)
  o3Value = pd.Series(o3Value)
  no2Value = pd.Series(no2Value)

  df = pd.DataFrame({ 'cityName': cityName, 'pm10Value': pm10Value, 'pm25Value': pm25Value, 'so2Value': so2Value, 'coValue': coValue, 'o3Value': o3Value, 'no2Value': no2Value})
  return df

In [ ]:
iface = gr.Interface(test, gr.inputs.Dropdown(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주', '세종']), outputs=['dataframe'])
iface.launch(debug=True) #dropdown은 항목 밑으로 나열하는 것

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://14609.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)
